In [27]:
# import packages:

import pandas as pd
import numpy as np
import os
import time as tm
import pandas_datareader as pdr
import yfinance as yf

import datetime
import requests_cache
import random
import glob

pd.set_option('max_rows', 100)
pd.set_option('max_columns', 100)

pd.options.mode.chained_assignment = None  # default='warn' -> setting this for the running total days -> works as expected, but doesn't like it


In [35]:
# functions:

def pull_stocks(stock_list, start_date, end_date, file_save_location, sleep_min = 2, sleep_max = 10, expire_days = 3):
    
    session = requests_cache.CachedSession(cache_name='cache', backend='sqlite', expire_after=expire_days)
    
    for stock in stock_list:
        
        start = tm.time()
        print('retrieving: ' , stock)
    
        ## pull and data generation:
        df = pdr.get_data_yahoo(stock.strip(), start = start_date, end = end_date)
        df['ticker'] = stock
        df = df.reset_index()
        df.to_csv(os.path.join(file_save_location, stock.strip() +'.csv'), index = False)
        print('saving: ', stock, ' data to: ', os.path.join(file_save_location, stock.strip() +'.csv'))

        end = tm.time()
        total = (end - start) / 60
        print(f"completed retrieving: {stock.strip()} data in: {total}")
        
        sleep_time = random.randint(2, 10)
        print(f"sleeping for: {sleep_time} seconds")
       

        # sleep between pulls so to not arouse suspicion:
        tm.sleep(sleep_time)

        del df, start, end, total, sleep_time
        
        
def fix_columns(dataframe: pd.DataFrame) -> list:
    
    '''function that takes a list of columns and modifies them to be easier to read -- assign to df.columns'''
    
    column_string_replace = ['\n','@',' ','__', '/', '-']

    columns = dataframe.columns

    columns = columns.map(lambda x: x.strip())
    columns = columns.map(lambda x : x.lower())

    for string in column_string_replace:
        columns = columns.map(lambda x : x.replace(string, '_') if isinstance (x, (str, bytes)) else x)

    return columns


def calculate_rolling_means(dataframe: pd.DataFrame, field: str, day_ranges: list, exponential = False) -> pd.DataFrame: 
    
    '''return a dataframe that includes all of the rolling means either straight or exponential appended to the overall dataset'''
    
    if exponential == True:
        
        for days in day_ranges:
            dataframe[str(days) +'_' + field + '_' + 'ema'] = dataframe[field].ewm(span = days, min_periods = days).mean()
    
    else:
        for days in day_ranges:
            dataframe[str(days) +'_' + field + '_' + 'sma'] = dataframe[field].rolling(days).mean()
        
    return dataframe


def calculate_rolling_std(dataframe: pd.DataFrame, field: str, day_ranges: list) -> pd.DataFrame: 
    
    '''return a dataframe that includes all of the rolling standard deviations of a given field'''
    
    for days in day_ranges:
        dataframe[str(days) +'_' + field + '_' + 'std'] = dataframe[field].rolling(days).std()

    return dataframe


def calculate_above_ma_measure(dataframe: pd.DataFrame, field: str,  compare_field_list: list, boolean: list = True) -> pd.DataFrame:
    
    '''returns boolean indicator variables for whether or not the current price field is above the reference value (e.g., moving average) OR pct difference to the reference value
    
    Args:
        dataframe: dataframe containing the target dataset
        field: price field for the reference value to be compared to (e.g., is current price above or below 7 sma)
        compare_list: string list of fields to be used to compare the current price to
        boolean: Argument that when = True returns boolean fields and when = False returns pct difference fields
    Returns:
        dataframe: dataframe with new fields appended to the end of the dataset
    '''
    
    if boolean == True:
        
        for compare_field in compare_field_list:
            dataframe['above_'+ compare_field + '_ind'] = np.where(dataframe[compare_field].isna(), np.nan, np.where(dataframe[field] > dataframe[compare_field], 1, 0))
    else:
        
        for compare_field in compare_field_list:
            dataframe[compare_field + '_pct_diff'] = np.where(dataframe[compare_field].isna(), np.nan, dataframe[field] / dataframe[compare_field] -1)
            
    return dataframe


def calculate_bollinger_bands(dataframe: pd.DataFrame, moving_avg_field: str, std_dev_field: str, 
                              num_std_dev: float, price_field: str,
                              return_top_distance: bool = True, return_bottom_distance: bool = True,
                              return_gap: bool = True
                             ) -> pd.DataFrame:
    
    ''' Function that calculated the upper and lower bollinger band values based on the moving average of the standard deviations supplied
    
    Args:
        dataframe: dataframe containing the target dataset
        moving_avg_field: moving average of the base stock price value to be used for comparisons
        std_dev_field: field containing the standard deviation of the target rolling period for the moving average field
        num_std_dev: the number of standard deviations to generate off of the band for the current value
        price_field: field representing the current price (note: must be same field from which SMA/EMA are based or results will be erroneous)
        return_top_distance: if true will return the current price distance (%) to the upper band
        return_bottom_distance: if true will return the current price distance (%) to the bottom band
        return_gap: gives both the range of the upper and lower bollinger bands as well as the range as a percent of the current price as new features in the dataframe
        
    Returns: 
        dataframe: dataframe that now contains the bollinger bands as columns in the dataset

    '''
    
    dataframe['upper_band'] = dataframe[moving_avg_field] + (num_std_dev * dataframe[std_dev_field])
    dataframe['lower_band'] = dataframe[moving_avg_field] - (num_std_dev * dataframe[std_dev_field])
    
    if return_top_distance == True:
        dataframe['bol_pct_from_top'] = dataframe[price_field] / dataframe['upper_band'] -1
                                                                                        
    if return_bottom_distance == True:
        dataframe['bol_pct_from_bottom'] = dataframe[price_field] / dataframe['lower_band'] -1
        
    if return_gap == True:
        dataframe['bol_range'] = dataframe['upper_band'] - dataframe['lower_band']
        dataframe['bol_range_pct'] = (dataframe['upper_band'] - dataframe['lower_band']) / dataframe[price_field]
        
    return dataframe


def cumulative_days_above_below_ma(dataframe: pd.DataFrame, ma_ind_fields: list, return_above: bool = True) -> pd.DataFrame:
    
    ''' Function that returns the running total number of days above or below a target moving average(s)
    
    Args:
        dataframe: dataframe containing the target dataset
        ma_fields: list of features representing different moving averages to be totalled over the index
        above_ma_ind_field: field of the indicator variable showing whether the current price at each date is above the moving average
        return_above: If set to true field created will be a running total of days above the target Moving averages, if False, will return running total days below the target moving averages
        
    Returns:
        dataframe: returns a dataframe with the newly generated fields appended 
        '''
    
    if return_above == True:
        for field in ma_ind_fields:
            dataframe['days_above_' + field] = dataframe.groupby(by = (dataframe[field] != dataframe[field].shift(1)).cumsum()).cumcount()+1
            dataframe['days_above_' + field] = np.where((dataframe[field] == 0) | dataframe[field].isna(), 0, dataframe['days_above_' + field] )
            
    else:
        for field in ma_ind_fields:
            dataframe['days_below_' + field] = dataframe.groupby(by = (dataframe[field] != dataframe[field].shift(1)).cumsum()).cumcount()+1
            dataframe['days_below_' + field] = np.where((dataframe[field] == 1) | dataframe[field].isna(), 0, dataframe['days_below_' + field] )  
    
    return dataframe


def create_base_target_features(dataframe: pd.DataFrame, target_field: str, target_lags: list, create_indicators: bool = True) -> pd.DataFrame:
    
    '''Function that generatesa series of target features that can be tested for significance against a hypothesis; will either return a % change from a baseline or an indicator for above or below
    
    Args:
        dataframe: dataframe containing the target dataset
        target_field: the field in the dataframe that is to be lagged and forecasted
        target_lags: list of lags to be generated as targets for the group in question
        create_indicators: if set to true, will return a set of indicator variables showing whether the target is above or below the prior-lag value
        
    Returns:
        dataframe: dataframe with newly generated target(s) appended to the datasets
    '''
    
    if create_indicators == True:
        
        for lag in target_lags:
            df['target_lag_ind_' + str(lag)] = np.where(dataframe[target_field].shift(periods = lag) > dataframe[target_field], 1, 0)
    
    else:
        for lag in target_lags:
            df['target_lag_' + str(lag)] = dataframe[target_field].shift(periods = lag)
            df['target_lag_' + str(lag) + '_pct'] = dataframe[target_field].shift(periods = lag) / dataframe[target_field]-1
            
    return dataframe

In [36]:
# pull the data down:
'''
start = '2018-01-01'
end = '2021-12-31'
file_location = '../data/01_raw/historic_data'

etfs = ['XLY', 'XLP', 'XLE', 'XLF', 'XLV', 'XLI', 'XLB', 'XLRE', 'XLK', 'XLU', 'SPY']

pull_stocks(stock_list = etfs, start_date = start, end_date = end, file_save_location = file_location)

'''

"\nstart = '2018-01-01'\nend = '2021-12-31'\nfile_location = '../data/01_raw/historic_data'\n\netfs = ['XLY', 'XLP', 'XLE', 'XLF', 'XLV', 'XLI', 'XLB', 'XLRE', 'XLK', 'XLU', 'SPY']\n\npull_stocks(stock_list = etfs, start_date = start, end_date = end, file_save_location = file_location)\n\n"

In [37]:
df = pd.read_csv('../data/01_raw/historic_data/XLU.csv')

df.columns = fix_columns(df)

In [38]:
day_ranges = [10, 20, 30, 50]
lags_used = [-30, -15, -10, -5, -1]

# calculate the moving averages:
df = calculate_rolling_means(dataframe = df, field = 'close', day_ranges = day_ranges, exponential = True)
df = calculate_rolling_means(dataframe = df, field = 'close', day_ranges = day_ranges, exponential = False)
df = calculate_rolling_std(dataframe = df , field = 'close', day_ranges = day_ranges)

# calculate roling standard deviations:
df = calculate_rolling_std(dataframe = df , field = 'close', day_ranges = day_ranges)

# calculate above or below MA/SMA:
df = calculate_above_ma_measure(dataframe = df, field = 'close', 
                                compare_field_list = ['10_close_sma', '20_close_sma', '30_close_sma', '50_close_sma'],
                                boolean = True)

# add in the moving average measures:
df = calculate_above_ma_measure(dataframe = df, field = 'close', 
                                compare_field_list = ['10_close_sma', '20_close_sma', '30_close_sma', '50_close_sma'],
                                boolean = False)

# add in the bollinger band features:
df = calculate_bollinger_bands(dataframe = df, moving_avg_field = '20_close_sma', std_dev_field = '20_close_std', 
                               num_std_dev = 2, price_field = 'close', return_top_distance = True, return_bottom_distance = True, return_gap = True)

# add in the days above or below a moving average:
df = cumulative_days_above_below_ma(dataframe = df, ma_ind_fields = ['above_10_close_sma_ind'], return_above = True)
df = cumulative_days_above_below_ma(dataframe = df, ma_ind_fields = ['above_10_close_sma_ind'], return_above = False)
df = cumulative_days_above_below_ma(dataframe = df, ma_ind_fields = ['above_20_close_sma_ind'], return_above = True)

# add in the target features:
df = create_base_target_features(dataframe = df, target_field = 'close', target_lags = lags_used, create_indicators = False)
df = create_base_target_features(dataframe = df, target_field = 'close', target_lags = lags_used, create_indicators = True)


# final step: drop nans:
df = df.dropna(how = 'any').reset_index(drop = True)


In [40]:
df.head()

,date,high,low,open,close,volume,adj_close,ticker,10_close_ema,20_close_ema,30_close_ema,50_close_ema,10_close_sma,20_close_sma,30_close_sma,50_close_sma,10_close_std,20_close_std,30_close_std,50_close_std,above_10_close_sma_ind,above_20_close_sma_ind,above_30_close_sma_ind,above_50_close_sma_ind,10_close_sma_pct_diff,20_close_sma_pct_diff,30_close_sma_pct_diff,50_close_sma_pct_diff,upper_band,lower_band,bol_pct_from_top,bol_pct_from_bottom,bol_range,bol_range_pct,days_above_above_10_close_sma_ind,days_below_above_10_close_sma_ind,days_above_above_20_close_sma_ind,target_lag_-30,target_lag_-30_pct,target_lag_-15,target_lag_-15_pct,target_lag_-10,target_lag_-10_pct,target_lag_-5,target_lag_-5_pct,target_lag_-1,target_lag_-1_pct,target_lag_ind_-30,target_lag_ind_-15,target_lag_ind_-10,target_lag_ind_-5,target_lag_ind_-1
0,2018-03-14,50.400002,49.840000,49.869999,50.230000,19132300.0,44.076794,XLU,49.575168,49.506200,49.540056,49.630654,49.434,49.4555,49.332000,49.9152,0.437472,0.513886,0.740249,1.004521,1.0,1.0,1.0,1.0,0.016102,0.015661,0.018203,0.006307,50.483272,48.427728,-0.005017,0.037216,2.055543,0.040923,4,0,5,51.189999,0.019112,50.820000,0.011746,50.259998,0.000597,49.540001,-0.013737,50.200001,-0.000597,1,1,1,0,0
1,2018-03-15,50.660000,49.900002,50.200001,50.200001,21067800.0,44.050465,XLU,49.688778,49.572680,49.584101,49.656318,49.547,49.5355,49.304000,49.8754,0.477145,0.497990,0.687417,0.950523,1.0,1.0,1.0,1.0,0.013179,0.013415,0.018173,0.006508,50.531481,48.539520,-0.006560,0.034209,1.991961,0.039680,5,0,6,51.709999,0.030080,50.419998,0.004382,50.529999,0.006574,49.759998,-0.008765,50.250000,0.000996,1,1,1,0,1
2,2018-03-16,50.380001,49.779999,49.810001,50.250000,19756300.0,44.436905,XLU,49.790821,49.637543,49.628445,49.682922,49.678,49.5640,49.304334,49.8448,0.471776,0.522407,0.687889,0.911796,1.0,1.0,1.0,1.0,0.011514,0.013841,0.019180,0.008129,50.608814,48.519186,-0.007090,0.035673,2.089629,0.041585,6,0,7,51.560001,0.026070,50.520000,0.005373,50.139999,-0.002189,49.060001,-0.023682,49.939999,-0.006169,1,1,0,0,0
3,2018-03-19,50.389999,49.650002,50.220001,49.939999,21689600.0,44.162766,XLU,49.817945,49.666492,49.649149,49.694378,49.678,49.5550,49.306334,49.8166,0.471776,0.513804,0.689705,0.885725,1.0,1.0,1.0,1.0,0.005274,0.007769,0.012852,0.002477,50.582608,48.527392,-0.012704,0.029109,2.055216,0.041154,7,0,8,51.400002,0.029235,50.130001,0.003805,50.310001,0.007409,49.570000,-0.007409,49.709999,-0.004606,1,1,1,0,0
4,2018-03-20,50.160000,49.590000,49.950001,49.709999,14968200.0,43.959373,XLU,49.798318,49.670654,49.653185,49.695071,49.722,49.5670,49.328000,49.7842,0.449488,0.514516,0.691906,0.858444,0.0,1.0,1.0,0.0,-0.000241,0.002885,0.007744,-0.001490,50.596033,48.537967,-0.017512,0.024147,2.058066,0.041401,0,1,9,51.349998,0.032991,50.060001,0.007041,50.419998,0.014283,50.279999,0.011466,49.540001,-0.003420,1,1,1,1,0


In [39]:
df[['close','target_lag_-30',	'target_lag_-15',	'target_lag_-10',	'target_lag_-5',	'target_lag_-1',	
    'target_lag_ind_-30',	'target_lag_ind_-15',	'target_lag_ind_-10',	'target_lag_ind_-5',	'target_lag_ind_-1']].head()

,close,target_lag_-30,target_lag_-15,target_lag_-10,target_lag_-5,target_lag_-1,target_lag_ind_-30,target_lag_ind_-15,target_lag_ind_-10,target_lag_ind_-5,target_lag_ind_-1
0,50.230000,51.189999,50.820000,50.259998,49.540001,50.200001,1,1,1,0,0
1,50.200001,51.709999,50.419998,50.529999,49.759998,50.250000,1,1,1,0,1
2,50.250000,51.560001,50.520000,50.139999,49.060001,49.939999,1,1,0,0,0
3,49.939999,51.400002,50.130001,50.310001,49.570000,49.709999,1,1,1,0,0
4,49.709999,51.349998,50.060001,50.419998,50.279999,49.540001,1,1,1,1,0


In [80]:
# create fields representing lagging close values: -> later

# create field for current status (backtesting strategy)
def moving_average_cross_backtest_status(dataframe: pd.DataFrame, price_field: str, slow_moving_avg_to_test: int, fast_moving_avg_to_test: int ) -> pd.DataFrame:
    
    '''Function that indicates whether a security should be bought, sold, or is currently held
    
    Args:
        dataframe: dataframe containing the target dataset
        slow_moving_avg_field: field with the longer of the 2 periods to be tested for a cross is contained
        fast_moving_avg_field: field with he shorter of the 2 periods to be tested for a cross is contained
    
    Returns:
        dataframe: dataframe containing a new column denoting the current status of the rule
        
    #TODO: WRITE UNIT TESTS AND EDGE CASES

    '''
    
    # calculate moving averages:
    dataframe[str(slow_moving_avg_to_test) + '_ma'] = dataframe[price_field].rolling(slow_moving_avg_to_test).mean()
    dataframe[str(fast_moving_avg_to_test) + '_ma'] = dataframe[price_field].rolling(fast_moving_avg_to_test).mean()
    
    dataframe['above_ind'] = np.where(dataframe[str(fast_moving_avg_to_test) + '_ma'] > dataframe[str(slow_moving_avg_to_test) + '_ma'], 1, 0)
    dataframe['days_of_cross'] = dataframe.groupby(by = (dataframe['above_ind'] != dataframe['above_ind'].shift(1)).cumsum()).cumcount()+1
    dataframe['ma_cross_strategy_action'] = np.where( (dataframe['above_ind'] == 1) & (dataframe['days_of_cross'] == 1), "Buy",
                                            np.where( (dataframe['above_ind'] == 1) & (dataframe['days_of_cross'] > 1) & (dataframe['above_ind'].index != 0), "Hold",
                                            np.where( (dataframe['above_ind'] == 0) & (dataframe['days_of_cross'] == 1) & (dataframe['above_ind'].index != 0), "Sell", "No action")))
    
    dataframe.drop(columns = [str(slow_moving_avg_to_test) + '_ma',str(fast_moving_avg_to_test) + '_ma'])

    return dataframe


#def moving_average_backtest_status(dataframe: pd.DataFrame, moving_average_field: str, price_field: str)


In [81]:
test = df[['date','close', '10_close_sma', '20_close_sma']]

test = moving_average_cross_backtest_status(dataframe = test, price_field = 'close', slow_moving_avg_to_test =  20, fast_moving_avg_to_test =  10)

test.head(40)

,date,close,10_close_sma,20_close_sma,20_ma,10_ma,above_ind,days_of_cross,ma_cross_strategy_action
0,2018-03-14,50.230000,49.434000,49.4555,NaN,NaN,0,1,No action
1,2018-03-15,50.200001,49.547000,49.5355,NaN,NaN,0,2,No action
2,2018-03-16,50.250000,49.678000,49.5640,NaN,NaN,0,3,No action
3,2018-03-19,49.939999,49.678000,49.5550,NaN,NaN,0,4,No action
4,2018-03-20,49.709999,49.722000,49.5670,NaN,NaN,0,5,No action
5,2018-03-21,49.540001,49.786000,49.6030,NaN,NaN,0,6,No action
6,2018-03-22,49.759998,49.836000,49.6375,NaN,NaN,0,7,No action
7,2018-03-23,49.060001,49.802000,49.5730,NaN,NaN,0,8,No action
8,2018-03-26,49.570000,49.798000,49.5415,NaN,NaN,0,9,No action
9,2018-03-27,50.279999,49.854000,49.5855,NaN,49.854000,0,10,No action


In [114]:
# write the function to put a valuation on the strategy:

def strategy_return(dataframe: pd.DataFrame, price_field: str, strategy_action_field: str, commission_per_trade: float, initial_investment: int = 10000)-> pd.DataFrame:
    
    '''function that takes the action column from the strategy to develop a valuation of the action over time
    
    Args: 
        dataframe: core data source for the valuation
        price_field: field used in making buy/sell decisions
        strategy_action_field: field representing a previously quantified strategy action (can take values-> buy, sell, hold, no-action)
        commission_per_trade: amount paid for each buy/sell transaction to be deducted from overall gain/loss
        initial_investment: amount to simulate investing over a time period
    
    Returns:
        new_dataframe: dataframe containing the totaled returns, number of transactions (buy/sell), average days held, return over baseline (buy + hold)
    
    #TODO: add in the returns for the index over the same period 
        
    '''
    
    # add in buy/sell rank field unique action for the action type:
    dataframe['action_rank'] = dataframe.groupby(strategy_action_field)['date'].rank(ascending=True)
    
    shares_transacted = []
    transacted_amount = []
    current_shares = 0 # instantiate at 0 value
    current_portfolio_value = initial_investment
    
    
    for index, row in dataframe.iterrows():
        
        if row[strategy_action_field] == 'No action':
     
            current_shares = 0
            shares_transacted.append(0)
            transacted_amount.append(0)
        
        elif (row[strategy_action_field] == 'Buy') & (row['action_rank'] == 1):
            current_shares = (initial_investment - commission_per_trade) / row[price_field]
            current_portfolio_value = (initial_investment - commission_per_trade)
            shares_transacted.append(current_shares)
        
        elif row[strategy_action_field] == 'Sell':
            current_portfolio_value = (current_shares * row[price_field]) - commission_per_trade
            shares_transacted.append(-current_shares) # doing this because this is the shares you are selling that you originally held
            current_shares = 0
            
        elif row[strategy_action_field] == 'Buy':
            current_shares = (current_portfolio_value - commission_per_trade) / row[price_field]
            current_portfolio_value = current_portfolio_value - commission_per_trade
            shares_transacted.append(current_shares)
            
        elif row[strategy_action_field] == 'Hold':
            shares_transacted.append(0)
        
        else: shared_transacted.append(99999999)
        
    
    return shares_transacted
            


In [120]:
test_2 = strategy_return(dataframe = test, price_field = 'close',strategy_action_field = 'ma_cross_strategy_action', commission_per_trade = 10.00, initial_investment = 5000)


In [121]:
test_2


[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 99.6803808052303,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 -99.6803808052303,
 0,
 98.59735040169586,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 -98.59735040169586,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 98.94801737349601,
 0,
 0,
 0,
 0,
 -98.94801737349601,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 92.8064385562183,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 -92.8064385562183,
 0,
 0,
 92.76131169082007,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 -92.76131169082007,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 91.64907658563313,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 -91.64907658563313,
 0,
 0,
 0,
 0

In [ ]:
# approach -> use backtesting strategy with rules in order to determine what the target feature should be